# LangGraph

In [1]:
import os

from langchain.schema import HumanMessage, AIMessage
from langchain_openai import AzureChatOpenAI

from dotenv import load_dotenv

load_dotenv()

os.environ["AZURE_OPENAI_API_KEY"] 
os.environ["AZURE_OPENAI_ENDPOINT"]
os.environ["TAVILY_API_KEY"]

llm = AzureChatOpenAI(
    openai_api_version="2023-07-01-preview",
    azure_deployment="gpt-4",
)

message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
llm([message])

AIMessage(content="J'adore la programmation.")

c:\Users\vaalt\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content="J'adore la programmation.")

In [10]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")


# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

In [11]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator


class AgentState(TypedDict):
   # The input string
   input: str
   # The list of previous messages in the conversation
   chat_history: list[BaseMessage]
   # The outcome of a given call to the agent
   # Needs `None` as a valid type, since this is what this will start as
   agent_outcome: Union[AgentAction, AgentFinish, None]
   # List of actions and corresponding observations
   # Here we annotate this with `operator.add` to indicate that operations to
   # this state should be ADDED to the existing values (not overwrite it)
   intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

In [12]:
from langchain_core.agents import AgentFinish
from langgraph.prebuilt.tool_executor import ToolExecutor

# This a helper class we have that is useful for running tools
# It takes in an agent action and calls that tool and returns the result
tool_executor = ToolExecutor(tools)

# Define the agent
def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}

# Define the function to execute tools
def execute_tools(data):
    # Get the most recent agent_outcome - this is the key added in the `agent` above
    agent_action = data['agent_outcome']
    output = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(output))]}

# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(data['agent_outcome'], AgentFinish):
        return "end"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"

In [13]:
from langgraph.graph import END, StateGraph

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [14]:
inputs = {"input": "what is the weather in sf", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'agent_outcome': AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'current weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'current weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}})])}
----
{'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'current weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'current weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}})]), "[{'url': 'https://en.climate-data.org/north-america/united-states-of-america/california/san-francisco-385/t/january-1/', 'content': 'San Fran

# Multiagent

In [2]:
import json

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    ChatMessage,
    FunctionMessage,
    HumanMessage,
)
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, StateGraph
from langgraph.prebuilt.tool_executor import ToolExecutor, ToolInvocation


def create_agent(llm, tools, system_message: str):
    """Create an agent."""
    functions = [format_tool_to_openai_function(t) for t in tools]

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards answering the question."
                " If you are unable to fully answer, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    return prompt | llm.bind_functions(functions)

In [13]:
from langchain_core.tools import tool
from typing import Annotated
from langchain_experimental.utilities import PythonREPL
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=5)


In [6]:
import operator
from typing import Annotated, List, Sequence, Tuple, TypedDict, Union

from typing_extensions import TypedDict


# This defines the object that is passed between each node
# in the graph. We will create different nodes for each agent and tool
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str

In [22]:
import functools


# Helper function to create a node for a given agent
def agent_node(state, agent, name):
    result = agent.invoke(state)
    # We convert the agent output into a format that is suitable to append to the global state
    if isinstance(result, FunctionMessage):
        pass
    else:
        result = HumanMessage(**result.dict(exclude={"type", "name"}), name=name)
    return {
        "messages": [result],
        # Since we have a strict workflow, we can
        # track the sender so we know who to pass to next.
        "sender": name,
    }

# Psychologist agent and node
psychologist_agent = create_agent(
    llm,
    [tavily_tool],
    system_message="An expert psychologist who seamlessly weaves neuroscientific theories into conversations, unraveling the complexities of human behavior and emotions.",
)
psychologist_node = functools.partial(agent_node, agent=psychologist_agent, name="Psychologist")

# Socioligist
sociologist_agent = create_agent(
    llm,
    [tavily_tool],
    system_message="A keen-eyed sociologist adept at dissecting societal patterns, investigating the collective psyche. Focus on group effects rather than individual effects.",
)
sociologist_node = functools.partial(agent_node, agent=sociologist_agent, name="Sociologist")

# Socioligist
economist_agent = create_agent(
    llm,
    [tavily_tool],
    system_message="A pragmatic economist who quantifies intangibles, connecting trends to economic implications with precision.",
)
economist_node = functools.partial(agent_node, agent=economist_agent, name="Economist")

In [23]:
tools = [tavily_tool]
tool_executor = ToolExecutor(tools)


def tool_node(state):
    """This runs tools in the graph

    It takes in an agent action and calls that tool and returns the result."""
    messages = state["messages"]
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    tool_input = json.loads(
        last_message.additional_kwargs["function_call"]["arguments"]
    )
    # We can pass single-arg inputs by value
    if len(tool_input) == 1 and "__arg1" in tool_input:
        tool_input = next(iter(tool_input.values()))
    tool_name = last_message.additional_kwargs["function_call"]["name"]
    action = ToolInvocation(
        tool=tool_name,
        tool_input=tool_input,
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(
        content=f"{tool_name} response: {str(response)}", name=action.tool
    )
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

In [25]:
# Either agent can decide to end
def router(state):
    # This is the router
    messages = state["messages"]
    last_message = messages[-1]
    if "function_call" in last_message.additional_kwargs:
        # The previus agent is invoking a tool
        return "call_tool"
    if "FINAL ANSWER" in last_message.content:
        # Any agent decided the work is done
        return "end"
    return "continue"

In [26]:
workflow = StateGraph(AgentState)

workflow.add_node("Psychologist", psychologist_node)
workflow.add_node("Sociologist", sociologist_node)
workflow.add_node("Economist", economist_node)
workflow.add_node("call_tool", tool_node)

workflow.add_conditional_edges(
    "Psychologist",
    router,
    {"continue": "Sociologist", "call_tool": "call_tool", "end": END},
)
workflow.add_conditional_edges(
    "Sociologist",
    router,
    {"continue": "Economist", "call_tool": "call_tool", "end": END},
)

workflow.add_conditional_edges(
    "Economist",
    router,
    {"continue": "Psychologist", "call_tool": "call_tool", "end": END},
)

workflow.add_conditional_edges(
    "call_tool",
    # Each agent node updates the 'sender' field
    # the tool calling node does not, meaning
    # this edge will route back to the original agent
    # who invoked the tool
    lambda x: x["sender"],
    {
        "Psychologist": "Psychologist",
        "Sociologist": "Sociologist",
        "Economist": "Economist"
    },
)
workflow.set_entry_point("Psychologist")
graph = workflow.compile()

In [27]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(
                content="Provide me with meaningful insights about the effects of social media on mental health, covering individual effects, collective effects and macroeconomics effects. Once you provided me with these three perpectives, finish."
            )
        ],
    },
    # Maximum number of steps to take in the graph
    {"recursion_limit": 150},
):
    print(s)
    print("----")

{'Psychologist': {'messages': [HumanMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "effects of social media on individual mental health"\n}', 'name': 'tavily_search_results_json'}}, name='Psychologist')], 'sender': 'Psychologist'}}
----
{'call_tool': {'messages': [FunctionMessage(content='tavily_search_results_json response: [{\'url\': \'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7785056/\', \'content\': \'Social Media and Mental Health: Benefits, Risks, and Opportunities for Research and Practice\\nJohn A. Naslund\\naDepartment of Global Health and Social Medicine, Harvard Medical School, Boston, MA\\nAmeya Bondre\\nbCareNX Innovations, Mumbai, India\\nJohn Torous\\ncDepartment of Psychiatry, Beth Israel Deaconess Medical Center, Boston, MA\\nKelly A. Aschbrenner\\ndDepartment of Psychiatry, Geisel School of Medicine at Dartmouth, Lebanon, NH\\nAbstract\\nSocial media platforms are popular venues for sharing personal experiences, seeking informa

In [28]:
outputs = []
for s in graph.stream(
    {
        "messages": [
            HumanMessage(
                content="Provide me with meaningful insights about the effects of social media on mental health, covering individual effects, collective effects and macroeconomics effects. Once you provided me with these three perpectives, finish."
            )
        ],
    },
    # Maximum number of steps to take in the graph
    {"recursion_limit": 150},
):
    print(s)
    outputs.append(s)
    print("----")

{'Psychologist': {'messages': [HumanMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "effects of social media on individual mental health"\n}', 'name': 'tavily_search_results_json'}}, name='Psychologist')], 'sender': 'Psychologist'}}
----
{'call_tool': {'messages': [FunctionMessage(content='tavily_search_results_json response: [{\'url\': \'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7785056/\', \'content\': \'Social Media and Mental Health: Benefits, Risks, and Opportunities for Research and Practice\\nJohn A. Naslund\\naDepartment of Global Health and Social Medicine, Harvard Medical School, Boston, MA\\nAmeya Bondre\\nbCareNX Innovations, Mumbai, India\\nJohn Torous\\ncDepartment of Psychiatry, Beth Israel Deaconess Medical Center, Boston, MA\\nKelly A. Aschbrenner\\ndDepartment of Psychiatry, Geisel School of Medicine at Dartmouth, Lebanon, NH\\nAbstract\\nSocial media platforms are popular venues for sharing personal experiences, seeking informa

In [37]:
print(outputs[-1]['__end__']['messages'][-1].content)

FINAL ANSWER:

Based on the search results, here are some insights about the effects of social media on mental health from three perspectives: individual, collective, and macroeconomic.

Individual effects: Social media platforms are popular venues for sharing personal experiences, seeking information, and offering peer-to-peer support among individuals living with mental illness. Many individuals living with mental disorders have expressed interest in using social media platforms for seeking mental health information, connecting with mental health providers, and accessing evidence-based mental health services delivered over social media specifically for coping with mental health symptoms or for promoting overall health and wellbeing. However, excessive use or problematic use of social media can have negative effects on mental health [source](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7785056/).

Collective effects: Some studies have explored the relationship between how individuals 